# Clasificador regresion lineal - hito 3

El siguiente .py contiene las configuraciones, las bases de dato procesadas y algunas funciones que se ocuparan para obtener resultados en el apartado de la pregunta número 3, el código en cuestión se encuentra en los anexos.

In [1]:
%%time
from config_p3 import *

Wall time: 50 s


El objetivo de este apartado es mostrar un método alternativo al obtenido en el primer clasificador para asociar palabras a indices de probabilidades de emojis.

**Diferencias en la base de datos**

La diferencia a mencionar al resto de iteraciones de entrenamiento que se han hecho en el transcurso de este informe es que para el caso de trabajar con una regresión lineal, esta no es capaz de entrenarse para lograr predecir varias clases al mismo tiempo, es por esto que es de suma importancia modificar la base de datos para que esta sea compatible con el clasificador, para esto se transformaran el atributo multiclase que determina que emoji es cada tweet, a varios atributos binarios donde cuando uno es “1” el resto es 0, de esta forma volviendo compatible la base, a continuación se puede ver una tabla donde se puede ver los cambios para ingles.

In [2]:
df_us_test2.head()

,id,text,label_0,label_1,label_10,label_11,label_12,label_13,label_14,label_15,...,label_19,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,tokenized_text
0,test0,en Pelham Parkway,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,en pelham parkway
1,test1,The calm before...... | w/ sofarsounds @user |...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,the calm before ... | w / sofarsounds @user | ...
2,test2,Just witnessed the great solar eclipse @ Tampa...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,just witnessed the great solar eclipse @ tampa...
3,test3,This little lady is 26 weeks pregnant today! E...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,this little lady is 26 weeks pregnant today ! ...
4,test4,"Great road trip views! @ Shartlesville, Pennsy...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"great road trip views ! @ shartlesville , penn..."


In [3]:
df_us_train2.head()

,id,text,label_0,label_1,label_10,label_11,label_12,label_13,label_14,label_15,...,label_19,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,tokenized_text
0,729044324441186304,Selfies for summatime @ Drexel University,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,selfies for summatime @ drexel university
1,663834134037442560,Ready to be a bulldog with rasso #hailstate #i...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,ready to be a bulldog with rasso #hailstate #i...
2,747449193350963200,#scored my new #matcotools #slidehammer weight...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,#scored my new #matcotools #slidehammer weight...
3,691439672761925637,@user last night was so much fun @ Skyway Thea...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,@user last night was so much fun @ skyway theatre
4,758118895618109440,love beach days @ Manasquan Beach,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,love beach days @ manasquan beach



Para las siguientes iteraciones de entrenamientos, se consideraron mínimos óptimos de palabras obtenidos en el primer clasificador multinomial naive bayes, donde para ingles el optimo es 5 palabras y 10 para español. 

Cabe destacar que el algoritmo de regresión lineal cuando genera una predicción que un emoji no pertenece a un tweet, este al ser de carácter binario, la regresión dice que es cualquier otro emoji, sin especificar cual, a diferencia del MulitinomialNB donde si se predice que un emoji no pertenece a cierto tweet, es porque el clasificador esta prediciendo que es otro en específico, lo que hace que comparar cual método es mejor, no es justo.

## Regresión lineal en ingles

Lo siguiente es un ejemplo de aplicar una regresión para el label del arbol de navidad, donde se va a buscar obtener los valores de los coeficientes de la regresión lineal, pero antes se va a analizar si la RL es capaz de predecir algo o no.

In [4]:
# Ejemplo de regresión lineal para el label 17
%time
reg.fit(X_train_bow, df_us_train2["label_17"])

Wall time: 0 ns


LinearRegression()

In [5]:
reg.score(X_train_bow, df_us_train2["label_17"])

0.4337708335088303

La regresión lineal cuando predice, entrega un número cercano entre 0 y 1, que consiste básicamente en la suma de los coeficientes de las palabras presentes en los tweets que se están prediciendo, por lo tanto, si se quieren utilizar métricas como por ejemplo: precision, recall o f1-score, es importante implementar una función de “cutoff” donde se va a considerar un umbral donde el algoritmo predijo “si”(1) o que “no”(0). Para que sea simple, se estimara que si la predicción (suma de coeficientes beta) de la regresión es mayor a 0,5 entonces se considera que, para ese tweet del test, existe suficiente seguridad para decir que el emoji corresponde al tweet.

In [7]:
y_pred = reg.predict(X_test_bow) # Ejemplo de lo que se obtiene la predecir con el cutoff en el label 17
Puntodecorte = 0.5 # punto del cutoff
y_predcut = cutoff(y_pred, Puntodecorte)
print(classification_report(df_us_test2["label_17"], y_predcut,output_dict = True ))
# Nos interesa la clase 1, ya que ahi se pone a prueba si se esta adivinando el emoji en cuestión

{'0': {'precision': 0.9804392029280196, 'recall': 0.995108863894335, 'f1-score': 0.9877195677779484, 'support': 48455}, '1': {'precision': 0.7109756097560975, 'recall': 0.3773462783171521, 'f1-score': 0.49302325581395345, 'support': 1545}, 'accuracy': 0.97602, 'macro avg': {'precision': 0.8457074063420585, 'recall': 0.6862275711057435, 'f1-score': 0.740371411795951, 'support': 50000}, 'weighted avg': {'precision': 0.9721127778990071, 'recall': 0.97602, 'f1-score': 0.9724334517382609, 'support': 50000}}


In [8]:
metrica_clase1_todos( X_train_bow , df_us_train2)     # Punto de corte 50%    

| ❤ | Precisión = 0.6235406602254429 | Recall = 0.1481982035856526 | f1-score = 0.2394789482330431 | ❤ |
None
| 😍 | Precisión = 0.680140597539543 | Recall = 0.03781697366492402 | f1-score = 0.07165008099976858 | 😍 |
None
| 😂 | Precisión = 0.7563370403767158 | Recall = 0.18687493811268444 | f1-score = 0.29970026003930367 | 😂 |
None
| 💕 | Precisión = 0.7111111111111111 | Recall = 0.006402881296583463 | f1-score = 0.0126914877794854 | 💕 |
None
| 🔥 | Precisión = 0.764804469273743 | Recall = 0.2732262249276519 | f1-score = 0.40261745459892645 | 🔥 |
None
| 😊 | Precisión = 0.8666666666666667 | Recall = 0.0028118747634239983 | f1-score = 0.005605562442731634 | 😊 |
None
| 😎 | Precisión = 0.7356948228882834 | Recall = 0.015764582238570676 | f1-score = 0.030867726077512286 | 😎 |
None
| ✨ | Precisión = 0.7250996015936255 | Recall = 0.039308855291576676 | f1-score = 0.07457488219627126 | ✨ |
None
| 💙 | Precisión = 0.7667560321715817 | Recall = 0.022587268993839837 | f1-score = 0.043881856540084384 

Se encuentran resultados mas altos de lo esperado en la precision, sin embargo el recall es muy bajo, por lo tanto el algoritmo de regresión lineal falla mucho en los falsos negativos ( Dice que no es el emoji, pero en realidad si es), a cambio de tener una precisión más alta. 

Para encontrar el top de palabras, vamos a predecir y luego este valor de predicción va a corresponder al valor que la regresión le asocio al coeficiente de cada regresión como ya se dijo en los parrafos anteriores

In [9]:
reg = LinearRegression()
reg.fit(X_train_bow, df_us_train2["label_17"])
vocab_length = X_train_bow.shape[1]

Wall time: 7.49 s


In [10]:
print(reg.coef_.size)
print(vocab_length)

17777
17777


Nos aseguramos de que el largo del vocabulario este en sintonía con la cantidad de coeficientes para tener evidencia de que el largo del vocabulario se relacióna con los coeficientes.

In [11]:
ejemplo_navidad_us()

🎄
(['ohchristmastree', 'holidaze', 'litmas', 'xmas2015', 'xmas2016'], array([0.54584545, 0.57313968, 0.62446983, 0.7050567 , 0.76373461]))


Apoyandonos con la métodologia propuesta, recordamos el modelo:


$$ Y1_{emoji}=β_{0}+β_{1} X_{1}+ ε $$ 

Donde Y es el emoji que se quiere predecir (variable binaria) 

Donde X nos dice si la palabra X esta presente en el tweet o no ( variable binaria) 

Donde β el valor probabilistico cuando esta la palabra X y como aporta en predecir Y  

Donde n es la cantidad de palabras que existen en todos los tweets 

Tomando el modelo Y1, vemos si utilizando los resultados obtenidos, es posible plantear el indice de probabilidad de cada palabra en un regresión, para esto se utlizara como ejemplo la palabra "litmas":

$$ Y1_{ArbolNavidad}=β_{Intercepto}+β_{litmas} X_{Contiene litmas?}+ ε $$ 
$$ Y1_{ArbolNavidad} = β_{Intercepto} + 0.6244698 * 1 +  ε $$

Si aproximamos el intercepto y el error a 0, entonces nos queda lo siguiente:

$$ Y1_{ArbolNavidad} ≈ 0.6244698 $$

Entonces si multiplicamos por 100 el coeficiente obtenemos finalmente que:

$$ Y1_{ArbolNavidad} ≈ 62.44 \% $$

Obteniendo de esta manera el indice de probabilidad asociado a una palabra para una regresión lineal. Lo siguiente es repetir el top de palabras para cada emoji de la base de datos, para de esta forma encontrar más indices de probabilidades asociados a palabras.

In [12]:
Listatop_us() # Función que entrega el TOP de palabras por emojis en us

❤
{'sibabes': 0.804948529378079, 'snowwhite': 0.8203815604017426, '1luv': 0.9551082292107682, 'cityofbrotherlylove': 1.0276042069771996, 'kfodiaries': 0.9732962170496406}
😍
{'asada': 0.6821768010788973, 'gyu': 0.8567913547961459, 'enamorada': 0.764880037442025, 'eatgood': 0.8798387923167201, 'encanta': 0.6974620745801943}
😂
{'lmfaoooo': 0.7756739280963982, 'foolin': 0.7836170807026172, 'hysterical': 0.8031867993103163, 'postavideoyoucantexplain': 0.9092025087300106, 'clownin': 0.980526728422956}
💕
{'froomies': 0.4184682101946456, 'twinny': 0.5270839756126672, 'endorsement': 0.5009280482968419, 'honeybglam': 0.5480836741094768, 'bakers': 0.42597044968754366}
🔥
{'linkinmybio': 0.7315935943422897, 'fiya': 0.748510479195848, 'feeltheburn': 0.8444973901079282, 'instant_classic': 0.845625695616753, 'onfire': 0.9160725933546516}
😊
{'iah': 0.35770813114790895, 'allah': 0.3617374629968806, 'snowglobe': 0.38610150714220465, 'trigger': 0.46180553488716364, 'goodtime': 0.528529456916411}
😎
{'sungl

Finalmente se encontro otro clasificador que puede relacionar cada palabra frecuente de un idioma a una probabilidad, si bien ya se habian encontrado probabilidades asociadas a palabras mediante el clasificador MultinomialNB, ahora se obtiene resultados mediante los coeficientes de la regresión lineal que relacionan a cada palabra con una probabilidad, lo siguiente es repetir el codigo para español.

## Regresión lineal en español

Se entreno el modelo en ingles, ahora se entrenara para español y se buscara y existe algunas diferencias relevantes o si esque tienene comportamientos similares

In [13]:
%%time
map_emojis_es = df_es_mapping["label"].values
metrica_clase1_todos_es( X_train_bow_es , df_es_train2)   # Punto de corte 50%    

| ❤ | Precisión = 0.7001746071339486 | Recall = 0.17432617066202957 | f1-score = 0.2791507135398538 | ❤ |
None
| 😍 | Precisión = 0.7580967612954818 | Recall = 0.16589377898328814 | f1-score = 0.27221823402727924 | 😍 |
None
| 😂 | Precisión = 0.823321554770318 | Recall = 0.24129449838187703 | f1-score = 0.3732105315847432 | 😂 |
None
| 💕 | Precisión = 0.8317757009345794 | Recall = 0.03328347045624533 | f1-score = 0.06400575332614168 | 💕 |
None
| 😊 | Precisión = 0.8777777777777778 | Recall = 0.072503671071953 | f1-score = 0.13394370973211256 | 😊 |
None
| 😘 | Precisión = 0.8726415094339622 | Recall = 0.050546448087431695 | f1-score = 0.09555785123966942 | 😘 |
None
| 💪 | Precisión = 0.8323765786452354 | Recall = 0.2320742637644046 | f1-score = 0.36295369211514394 | 💪 |
None
| 😉 | Precisión = 0.8893805309734514 | Recall = 0.06448508180943215 | f1-score = 0.12025127131319174 | 😉 |
None
| 👌 | Precisión = 0.945054945054945 | Recall = 0.029819694868238558 | f1-score = 0.05781512605042017 | 👌 |
No

Se encuentran los mismos resultados que en ingles, una precision alta a cambio de un recall muy bajo

In [14]:
Listatop_es() # Función que entrega el TOP de palabras por emojis en es

❤
{'weddingdress': 1.0719928400095697, 'guapet': 1.1680646749089834, 'modeloporundia': 1.1946272585010524, 'waterpolo': 1.2259800778188212, 'odon': 1.3557269237896223}
😍
{'alucinante': 0.7972977290503844, 'ricuras': 0.8068529342025612, 'amordehermanos': 0.8511529327936498, 'sinpalabras': 0.8738627220003872, 'losamo': 1.1657874878322254}
😂
{'jajajajajajajajajaja': 0.9334058630660954, 'gelida': 0.9436811268432989, 'gelidamenamora': 0.9563973975124452, 'postureomaximo': 1.0545066911169394, 'matata': 0.987014013449593}
💕
{'teamooooo': 0.5821202017491293, 'distinta': 0.5892009737114164, 'arganzuela': 0.6570108954514027, 'fortuny': 0.6689502521598631, 'valdivia': 0.6781009146416713}
😊
{'seguirnos': 0.7749594547176322, 'esmorzant': 0.7957428754906615, 'thediyingroom': 0.7966060206892784, 'kikejaen': 0.8263083920231762, 'askdavidparejo': 0.8398547881346923}
😘
{'guapaaaa': 0.5643463573550496, 'besis': 0.5800697772566662, 'mai': 0.5987995214884582, 'wlwspain': 0.6987068011803832, 'guapaaa': 0.72

Finalmente se obtiene las palabras que son más frecuentes según que emoji en español, a continuación, se desarrollaron tablas resumen que incluyen las 3 palabras más probables que estén asociadas a un emoji para el caso de la Naive-Bayes ( Que se de desarrollo en la primera pregunta) y regresión lineal.

**Tabla X.** *Ejemplos de Top de palabras por emoji en ingles 🇺🇸 *

|       Emoji    |   Palabra    |  Probabilidad    |   Naive-Bayes    | Regresión lineal| 
|----------------|---------------------|---------|-------|------------|
|       😂       |      lmfao                     |  0.884 | ✔️    | |
|       😂       |      postavideoyoucantexplain  |  0.901 | ✔️    | |  
|       😂       |      lmaooo                    |  0.872 | ✔️    | |
|       😂       |      clownin                   |  0.980 |    | ✔️ |
|       😂       |      postavideoyoucantexplain  | 0.909  |    | ✔️ |
|       😂       |      hysterical                |  0.803 |    | ✔️ |
|       🎄       |      christmastree                     |  0.846 | ✔️    | |
|       🎄       |      ohchristmastree |  0.845 | ✔️    | |  
|       🎄       |      xmas2016                   |  0.828 | ✔️    | |
|       🎄       |      xmas2016                  |  0.763|    | ✔️ |
|       🎄       |      xmas2015  | 0.705 |    | ✔️ |
|       🎄       |      litmas               |  0.624|    | ✔️ |
|       🔥       |      flames                    |  0.835 | ✔️    | |
|       🔥       |      fuego |  0.823| ✔️    | |  
|       🔥       |      instant_classic                  |  0.778| ✔️    | |
|       🔥       |      onfire                 |  0.916|    | ✔️ |
|       🔥       |      instant_classic  | 0.845|    | ✔️ |
|       🔥       |      feeltheburn              |  0.844|    | ✔️ |



**Tabla X.** *Ejemplos de Top de palabras por emoji en español 🇪🇸 *

|       Emoji    |   Palabra    |  Probabilidad    |   Naive-Bayes    | Regresión lineal| 
|----------------|---------------------|---------|-------|------------|
|       😎       |      worldwide                     |  0.641 | ✔️    | |
|       😎       |     matando |  0.636 | ✔️    | |  
|       😎       |      sunglasses                  |  0.692| ✔️    | |
|       😎       |      domicilio                |  0.625|    | ✔️ |
|       😎       |      elguay  | 0.660 |    | ✔️ |
|       😎       |      optico              |  0.611|    | ✔️ |
|       💪       |     empujón                   |  0.693 | ✔️    | |
|       💪       |      altafit |  0.670 | ✔️    | |  
|       💪       |      gymlife                |  0.664| ✔️    | |
|       💪       |      powerlifting                |  0.842|    | ✔️ |
|       💪       |     quiendijomiedo | 0.996 |    | ✔️ |
|       💪       |      vamooos              |  1.006|    | ✔️ |
|       😍       |      losamo                   |  0.684 | ✔️    | |
|       😍       |      preciosidad |  0.643| ✔️    | |  
|       😍       |      amordehermanos                 |  0.626| ✔️    | |
|       😍       |      amordehermanos               |  0.851|    | ✔️ |
|       😍       |      sinpalabras | 0.873    | ✔️ |
|       😍       |      losamo             |  1.165|    | ✔️ |




Podemos ver algunos patrones presentes en la tabla de ingles donde para el emoji 😂 "postavideoyoucantexplain" tiene con porcentaje muy parecido en ambos ambos clasificadores esto también ocurre para el emoji 🔥 donde instant_classic se repiten pero no con una probabilidad tan cercana, notamos que en ambos casos se repite que sea una palabra compuesta.    

Para español vemos que ocurre algo interesante con el emoji 💪 donde la palabra “vamos” tiene un coeficiente mayor 1, al igual que con el emoji 😍 donde “losamos” ocurre lo mismo, significando que hay extrema seguridad que al poner estas palabras encontraremos estos emojis.

Finalmente podemos concluir que si se puede asociar una palabra a un índice de probabilidad a un emoji especifico, es más, se lograron obtener por medio de dos clasificadores diferentes. Si se quiere expandir esta pregunta es posible generar ensamblaje de clasificadores, es decir "mezclar" las predicciones de varios algoritmos, donde para el contexto de esta pregunta seria parecido a promediar la probabilidad de cada palabra que entregue cada clasificador. Por último, otro acercamiento que se le pudo haber dado es la búsqueda de pares de palabras con los índices más altos de probabilidad según el emoji, y comparar estos resultados con los obtenidos al buscar índices de palabras únicas.
